## *Important note
Please ensure that you installed all the required python pip libraries as described in the REAME.md File.
- It is recommended to create a virtual environment and install pip environment using the following command:

```bash
pip install -r pip_requirements.txt
```
- Alteratively, you can install all required pip libraries individually

# Import Statements

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import torchvision.models as models
from torch.utils.data import random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from utils import GardynDataset, envDataset, tensor_toImage, initiate_dir, save_model_config, load_model_config, imshow_batch, denormalize_image_tensor, normalize_image_tensor,calculate_fid, save_metrics, images_to_video
torch.set_printoptions(precision=4, threshold=10, edgeitems=10, linewidth=100)
from scipy import linalg
from PIL import Image
from models import SI_PGS, SI_PGS_R, cGAN

import numpy as np
np.set_printoptions(precision=4, threshold=10, edgeitems=10, linewidth=100, suppress=True)

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import os
from tqdm.notebook import tqdm, trange
import gc

import time, datetime, pytz
str_format = '%Y%m%d%H%M%S'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Available Device: {device}')
feature_encoder=False #Do not change this

# USER INPUT

In [ ]:
## User input
output_dir = './output/'
input_file = './example_data.csv' ## path to CSV File with environmental data as inputs
trial_name = 'test1' #name this run or outputs will be overwritten


### Uncomment the model you would like to test

In [ ]:
#uncomment this section for SI-PGS

model = 'SI-PGS'
feature_encoder, generator, discriminator= SI_PGS(beta='0.75', device=device) # available beta = 0.25, 0.50, 0.75. 0.90

In [ ]:
#uncomment this section for SI-PGS

# model = 'SI-PGS-R'
# feature_encoder, generator, discriminator= SI_PGS_R(device=device)

In [ ]:
#uncomment this section for cGAN

# model = 'cGAN'
# generator, discriminator= cGAN(device=device)

##  Restart Kernel and Run all Cells! 

Click this button above

![](./figures/restartandrun.png)

Your results will be saved in the output folder that you specified under the trial name!

# Load data (Do not change)

In [ ]:
# Data Paths
initiate_dir(output_dir)

##Output directories
sim_out_dir = f'{output_dir}{trial_name}_simulated_output/'
initiate_dir(sim_out_dir, True)


In [ ]:
data = envDataset(f'{input_file}', normalized =True, cumulative = True, params = ['Temperature_C', 'Humidity_percent', 'EC', 'PH', 'WaterTemp_C'])
data_loader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False)
dataiter = iter(data_loader)

# Run the model (Do not change)

In [ ]:
if model == 'SI-PGS':
    with torch.no_grad():
        i=0
        z_cs = torch.randn(1,generator.latent_size).to(device)
        
        feature_encoder.eval(), generator.eval(), discriminator.eval()
        for label, sequence_length, ts in tqdm(data_loader):
            
            label = label.to(device)
            
            ## Encoder sequential feature data
            feature_embedding = feature_encoder(label, sequence_length)
    
            y_hat = generator.decode(z_cs, feature_embedding)
    
            y_out = y_hat[0].detach().cpu().numpy()
                
            img = np.transpose(y_out, (1, 2, 0))
            img = Image.fromarray((img*255).astype(np.uint8))
            img.save(f'{sim_out_dir}{i}.jpg')
    
            i+=1
            
elif model == 'SI-PGS-R':
    img_path = './first_frame.jpg'
    transform = transforms.Compose([
            transforms.ToTensor(),  # Convert PIL images to tensors
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    image = Image.open(img_path).convert('RGB')
    image_normalized_prior  = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        i=0
        z_cs = torch.randn(1,generator.latent_size).to(device)
        ts_prior = torch.tensor(0)
        
        feature_encoder.eval(), generator.eval(), discriminator.eval()
        for label, sequence_length, ts in tqdm(data_loader):

            time_diff = ts - ts_prior
            time_diff = time_diff.unsqueeze(1).to(device)
            
            label = label.to(device)
            
            ## Encoder sequential feature data
            feature_embedding = feature_encoder(label, sequence_length)
    
            y_hat = generator.decode(z_cs, feature_embedding,image_normalized_prior, time_diff)
    
            y_out = y_hat[0].detach().cpu().numpy()

            image_normalized_prior = normalize_image_tensor(y_hat)
            ts_prior = ts
                
            img = np.transpose(y_out, (1, 2, 0))
            img = Image.fromarray((img*255).astype(np.uint8))
            img.save(f'{sim_out_dir}{i}.jpg')
    
            i+=1

elif model == 'cGAN':

    data = envDataset(f'{input_file}', normalized =True, cumulative = False, params = ['Temperature_C', 'Humidity_percent', 'EC', 'PH', 'WaterTemp_C'])
    data_loader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False)
    dataiter = iter(data_loader)
    
    with torch.no_grad():
        i=0
        noise = torch.randn(1,generator.noise_dim, device=device)
        
        generator.eval(), discriminator.eval()
        for label, ts in tqdm(data_loader):
            
            label = label.to(device)    
            y_hat = generator(noise, label)
    
            y_out = y_hat[0].detach().cpu().numpy()
                
            img = np.transpose(y_out, (1, 2, 0))
            img = Image.fromarray((img*255).astype(np.uint8))
            img.save(f'{sim_out_dir}{i}.jpg')
    
            i+=1

else:
    print('invalid Model')



## Save Video (Do not change)

In [ ]:
images_to_video(sim_out_dir, f'{output_dir}{trial_name}.mp4',120)